In [1]:
import os
import pandas as pd
import arcpy
import datetime

arcpy.env.overwriteOutput = True

Script to preprocess earth quake data for a 3d point density demonstration.


In [2]:
# Set up path variables

base_dir = r'C:\test\blog\3dPointDensity\eq'
os.chdir(base_dir)
gdb_name = 'Earthquake_density.gdb'
base_gdb = os.path.join(base_dir, gdb_name)

# If it exists, delete the old working geodatabase for the script
if arcpy.Exists(base_gdb):
    arcpy.Delete_management(base_gdb)

# Create a geodatabase for the script to work in
if not arcpy.Exists(base_gdb):
    arcpy.management.CreateFileGDB(base_dir, gdb_name, "CURRENT")
    
    

In [3]:
# List files in the working folder
os.listdir()

['.ipynb_checkpoints',
 'Earthquake_density.gdb',
 'Oaklahoma.xlsx',
 'Oaklahoma_10000m.swm',
 'Oaklahoma_EQ.csv',
 'Oaklahoma_EQ_0.csv',
 'Process_EQ.ipynb']

# Data Preparation

You can download the data I am using as a csv from http://angp.maps.arcgis.com/home/item.html?id=c4a6a3bbc45748fa9b264e680a981cf5#overview

In [4]:
# Read the data into a pandas dataframe
df = pd.read_csv('Oaklahoma_EQ_0.csv')

In [5]:
# Having a first look at the data
df.head()

,Year?,Month?,Day?,Time UTC,Mag?,Lat,Lon,Depth_km,Region,IRIS_ID,Timestamp,FID,x,y
0,2016,12,3,7/29/2017 6:28:11 AM,2.7,36.24,-97.00,1.3,OKLAHOMA,5199564,1480746491,1,-97.00,36.24
1,2016,12,3,7/29/2017 2:21:55 AM,2.4,35.84,-97.38,5.5,OKLAHOMA,5199725,1480731715,2,-97.38,35.84
2,2016,12,1,7/29/2017 2:43:58 PM,3.0,36.45,-98.73,5.0,OKLAHOMA,5199420,1480603438,3,-98.73,36.45
3,2016,12,1,7/29/2017 5:16:28 AM,2.4,34.99,-97.88,7.9,OKLAHOMA,5199384,1480569388,4,-97.88,34.99
4,2016,12,1,7/29/2017 12:05:30 AM,2.7,35.99,-96.79,6.3,OKLAHOMA,5199351,1480550730,5,-96.79,35.99


In [6]:
# Remove ? from the column names
columns_rename = []
for idx, column in enumerate(df.columns):
    columns_rename.append(column.replace("?", ""))
df.columns = columns_rename

In [7]:
df.describe()

,Year,Month,Day,Mag,Lat,Lon,Depth_km,IRIS_ID,Timestamp,FID,x,y
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1000.000000
mean,2016.714000,5.856000,15.264000,2.538700,36.490910,-97.505520,5.400700,9.072625e+06,1.488174e+09,500.500000,-97.505520,36.490910
std,0.452115,3.617506,8.591048,0.560077,0.548822,1.152305,1.786298,1.977131e+06,7.061316e+06,288.819436,1.152305,0.548822
min,2016.000000,1.000000,1.000000,0.500000,34.600000,-99.640000,1.000000,5.195462e+06,1.476137e+09,1.000000,-99.640000,34.600000
25%,2016.000000,3.000000,8.000000,2.200000,36.167500,-98.000000,4.500000,9.948631e+06,1.482225e+09,250.750000,-98.000000,36.167500
50%,2017.000000,5.000000,15.000000,2.600000,36.445000,-97.650000,5.000000,1.000670e+07,1.488712e+09,500.500000,-97.650000,36.445000
75%,2017.000000,10.000000,23.000000,2.800000,37.000000,-97.190000,6.400000,1.015701e+07,1.494476e+09,750.250000,-97.190000,37.000000
max,2017.000000,12.000000,31.000000,5.000000,38.950000,-91.390000,14.300000,1.027615e+07,1.500162e+09,1000.000000,-91.390000,38.950000


The data has both a Year, Month, Day column and a [unix timestamp](http://www.unixtimestamp.com). I want to convert
this into a full date time field for my analysis

In [8]:
def timestamp_to_date(x):
    """Funciton to convert time stamp into formatted date
    The time stamp is the number of seconds since Jan 01 1970. (UTC)"""
    return datetime.datetime.fromtimestamp(
        int(x['Timestamp'])
    ).strftime('%Y-%m-%d %H:%M:%S')

In [9]:
# Test the timestamp function on a date
print("Row: \n")
print(df[:1])
print("\nConverted time stamp: \n")
print(timestamp_to_date(df[:1]))

Row: 

   Year  Month  Day              Time UTC  Mag    Lat   Lon  Depth_km  \
0  2016     12    3  7/29/2017 6:28:11 AM  2.7  36.24 -97.0       1.3   

     Region  IRIS_ID   Timestamp  FID     x      y  
0  OKLAHOMA  5199564  1480746491    1 -97.0  36.24  

Converted time stamp: 

2016-12-02 22:28:11


The time stamp conversion looks like it worked.

In [10]:
#####
# Data preparation

# Create a new row by applying the timestamp formula to the dataframe
df['Date'] = df.apply(timestamp_to_date, axis=1)

# Change depth to a negative number so it is below ground
df['Depth_km'] = -df['Depth_km']

# Create a depth in meters field
df['Depth_m'] = df['Depth_km'] * 1000

# Create a unique ID column
df['MYID'] = df.index

In [11]:
# Final look at the data
df.head()

,Year,Month,Day,Time UTC,Mag,Lat,Lon,Depth_km,Region,IRIS_ID,Timestamp,FID,x,y,Date,Depth_m,MYID
0,2016,12,3,7/29/2017 6:28:11 AM,2.7,36.24,-97.00,-1.3,OKLAHOMA,5199564,1480746491,1,-97.00,36.24,2016-12-02 22:28:11,-1300.0,0
1,2016,12,3,7/29/2017 2:21:55 AM,2.4,35.84,-97.38,-5.5,OKLAHOMA,5199725,1480731715,2,-97.38,35.84,2016-12-02 18:21:55,-5500.0,1
2,2016,12,1,7/29/2017 2:43:58 PM,3.0,36.45,-98.73,-5.0,OKLAHOMA,5199420,1480603438,3,-98.73,36.45,2016-12-01 06:43:58,-5000.0,2
3,2016,12,1,7/29/2017 5:16:28 AM,2.4,34.99,-97.88,-7.9,OKLAHOMA,5199384,1480569388,4,-97.88,34.99,2016-11-30 21:16:28,-7900.0,3
4,2016,12,1,7/29/2017 12:05:30 AM,2.7,35.99,-96.79,-6.3,OKLAHOMA,5199351,1480550730,5,-96.79,35.99,2016-11-30 16:05:30,-6300.0,4


In [12]:
# Save the processed data to a csv
eq_csv = "Oaklahoma_EQ.csv"
df.to_csv(eq_csv)

# Spatial Data Analysis
Now that the data is prepared I will create a feature class using the csv and start working on the spatial data analysis

In [13]:
csv_full_path = os.path.join(base_dir, eq_csv)

# Make an X Y event layer out of the csv using latitude, longitude and depth
eq_layer_name = "Oaklahoma_eq"
arcpy.management.MakeXYEventLayer(csv_full_path, "Lon", "Lat", eq_layer_name, 
                                  "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],VERTCS['WGS_1984_Geoid',VDATUM['WGS_1984_Geoid'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119522E-09;0.001;0.001;IsHighPrecision",
                                  "Depth_m")


<Result 'Oaklahoma_eq'>

In [14]:
# Copy the event layer into a feature
eq_feature = os.path.join(base_gdb, eq_layer_name)
arcpy.management.CopyFeatures(eq_layer_name, eq_feature, None, None, None, None)


<Result 'C:\\test\\blog\\3dPointDensity\\eq\\Earthquake_density.gdb\\Oaklahoma_eq'>

In [15]:
#Reproject to an equal area projection
eq_feature_proj_name = "{}_Proj".format(eq_layer_name)
eq_feature_proj_path = os.path.join(base_gdb, eq_feature_proj_name)

transform_string = "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],VERTCS['WGS_1984_Geoid',VDATUM['WGS_1984_Geoid'],PARAMETER['Vertical_Shift',0.0],PARAMETER['Direction',1.0],UNIT['Meter',1.0]]"
arcpy.management.Project(r"C:\test\blog\3dPointDensity\eq\Earthquake_density.gdb\Oaklahoma_eq", r"C:\test\blog\3dPointDensity\eq\Earthquake_density.gdb\Oaklahoma_eq_Proj", "102003", "WGS_1984_(ITRF00)_To_NAD_1983", transform_string, "NO_PRESERVE_SHAPE", None, "VERTICAL")

print(eq_feature_proj_path)

C:\test\blog\3dPointDensity\eq\Earthquake_density.gdb\Oaklahoma_eq_Proj



### Spatial Weights Matrix

Now the input data is ready as a feature class with x, y and z coordinates. The next step is to create a spatial weights matrix. Using a fixed distance band the tool will count all of the spatial neighbours within the threshold. Since the distance is fixed it will be possible to compare how many neighbours each point has.

In [16]:
# Create spatial weights matrix
swm_name = "Oaklahoma_10000m.swm"
swm_path = os.path.join(base_dir, swm_name)
if os.path.exists(swm_path):
    os.remove(swm_path)
r1 = arcpy.stats.GenerateSpatialWeightsMatrix(eq_feature_proj_path, "MYID", 
                                         swm_path, "FIXED_DISTANCE", "EUCLIDEAN", 1, 10000, 0, 
                                         "NO_STANDARDIZATION", None, None, None, None, "USE_Z_VALUES")
# Convert SWM to table
swm_table = "Oaklahoma_10000m"
swm_table_fullpath = os.path.join(base_gdb, swm_table)
arcpy.stats.ConvertSpatialWeightsMatrixtoTable(swm_path, swm_table_fullpath)

#Make a feature layer and a table view for the points and swm table
eq_layer_name = "Oaklahoma_EQ_Neighbours"
arcpy.MakeFeatureLayer_management(eq_feature_proj_path, eq_layer_name)
table_view_name = "Oaklahoma_neighbour_swm"
arcpy.MakeTableView_management(swm_table_fullpath, table_view_name)

# Add join to the original data and the table
arcpy.management.AddJoin(eq_layer_name, "MYID", table_view_name, "MYID", "KEEP_ALL")
# arcpy.management.AddJoin(r"C:\test\blog\3dPointDensity\3dPointDensity.gdb\Oaklahoma_eq", "MyID", r"C:\test\blog\3dPointDensity\3dPointDensity.gdb\Oaklahoma_eq_swm", "MyID", "KEEP_ALL")

# Copy the joined table to a new file
eq_joined_pre_agg = "Oakalhoma_neighbour_pre_agg"
eq_joined_pre_agg_fullpath = os.path.join(base_gdb, eq_joined_pre_agg)
arcpy.management.CopyFeatures(eq_layer_name, eq_joined_pre_agg_fullpath, None, None, None, None)
print(eq_joined_pre_agg_fullpath)

C:\test\blog\3dPointDensity\eq\Earthquake_density.gdb\Oakalhoma_neighbour_pre_agg


In [17]:
# Check the detailed result messages
for i in range(r1.messageCount):
    print(r1.getMessage(i))

Start Time: Saturday, July 29, 2017 1:29:58 PM
Running script GenerateSpatialWeightsMatrix...
Constructing spatial weights based on distance criteria....
WARNING 000846: 22 feature(s) had no neighbors which generally invalidates the statistical properties of a test.
WARNING 000847: Features with no neighbors (only includes first 30): MYID = 3, 61, 75, 170, 228, 273, 374, 435, 451, 489, 598, 615, 665, 669, 777, 793, 851, 877, 878, 942, 954, 964.

      Spatial Weights Matrix Summary     
Number of Features:                  1000 
Percentage of Spatial Connectivity:  2.73 
Average Number of Neighbors:         27.27
Minimum Number of Neighbors:         0    
Maximum Number of Neighbors:         70   

Distance measured in Meters

Completed script GenerateSpatialWeightsMatrix...
Succeeded at Saturday, July 29, 2017 1:29:59 PM (Elapsed Time: 1.00 seconds)


In [18]:
# The join has create a many to many relationship in our table, there is a row for each MYID and
# each of its neighbours, so if MYID 1 has 20 neighbours, there are 20 rows wiht MYID 1
# To aggregate the data it is possible to use summary statistics
summary_table = "Oaklahoma_eq_summary"
summary_table_fullpath = os.path.join(base_gdb, summary_table)
arcpy.analysis.Statistics(eq_joined_pre_agg_fullpath, summary_table_fullpath, "Oaklahoma_10000m_WEIGHT SUM", 
                          "Oaklahoma_eq_Proj_MYID")


# Join the summary statistics back to the input points
eq_input_layer = "Oaklahoma_eq_input"
arcpy.MakeFeatureLayer_management(eq_feature_proj_path, eq_input_layer)
summary_table_view = "Oaklahoma_summary_table"
arcpy.MakeTableView_management(summary_table_fullpath, summary_table_view)

# Join the two data sets
arcpy.management.AddJoin(eq_input_layer, "MYID", summary_table_view, "Oaklahoma_eq_Proj_MYID", "KEEP_ALL")

# Copy the results to an output feature class
eq_output_final = "Oaklahoma_eq_Final"
eq_output_final_fullpath = os.path.join(base_gdb, eq_output_final)
arcpy.management.CopyFeatures(eq_input_layer, eq_output_final_fullpath, None, None, None, None)

# Add the X Y Z attributes to the final output data to check the coordinates look correct
arcpy.management.AddXY(eq_output_final_fullpath)

# OK! Now we are ready to visualize
print(eq_output_final_fullpath)
print("Data processing complete")


C:\test\blog\3dPointDensity\eq\Earthquake_density.gdb\Oaklahoma_eq_Final
Data processing complete


[Results Feature Service](https://www.arcgis.com/home/item.html?id=82995f80f5ec4009bb9ff56730ba9a1a/0)

[3d Web Scene](http://angp.maps.arcgis.com/home/webscene/viewer.html?webscene=f0249df74b6d4f6dbc5ade6d06618e4f)

